In [2]:
import pandas as pd

In [4]:
delhi = pd.read_csv("data/Indian_housing_Delhi_data.csv")
mumbai = pd.read_csv("data/Indian_housing_Mumbai_data.csv")
pune = pd.read_csv("data/Indian_housing_Pune_data.csv")
# data

In [5]:
delhi["city"] = "Delhi"
mumbai["city"] = "Mumbai"
pune["city"] = "Pune"

In [6]:
df = pd.concat([delhi, mumbai, pune], ignore_index=True)

In [7]:
df.shape
df.info()
df.head()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13910 entries, 0 to 13909
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   house_type        13910 non-null  object 
 1   house_size        13910 non-null  object 
 2   location          13910 non-null  object 
 3   city              13910 non-null  object 
 4   latitude          13910 non-null  float64
 5   longitude         13910 non-null  float64
 6   price             13910 non-null  int64  
 7   currency          13910 non-null  object 
 8   numBathrooms      13854 non-null  float64
 9   numBalconies      5291 non-null   float64
 10  isNegotiable      1276 non-null   object 
 11  priceSqFt         0 non-null      float64
 12  verificationDate  13910 non-null  object 
 13  description       13079 non-null  object 
 14  SecurityDeposit   13910 non-null  object 
 15  Status            13910 non-null  object 
dtypes: float64(5), int64(1), object(10)
memo

house_type              0
house_size              0
location                0
city                    0
latitude                0
longitude               0
price                   0
currency                0
numBathrooms           56
numBalconies         8619
isNegotiable        12634
priceSqFt           13910
verificationDate        0
description           831
SecurityDeposit         0
Status                  0
dtype: int64

In [8]:
df["house_size"] = (
    df["house_size"]
    .str.replace("sq ft", "", regex=False)
    .str.strip()
    .astype(float)
)


ValueError: could not convert string to float: '1,020'

In [9]:
# 1. Convert to string (safety)
df["house_size"] = df["house_size"].astype(str)

# 2. Remove 'sq ft'
df["house_size"] = df["house_size"].str.replace("sq ft", "", regex=False)

# 3. Remove commas like '1,020'
df["house_size"] = df["house_size"].str.replace(",", "", regex=False)

# 4. Strip extra spaces
df["house_size"] = df["house_size"].str.strip()

# 5. Convert to numeric safely
df["house_size"] = pd.to_numeric(df["house_size"], errors="coerce")


In [10]:
df["house_size"].isna().sum()
df["house_size"].describe()


count    13910.000000
mean      1722.942703
std       1672.692701
min        150.000000
25%        720.000000
50%       1050.000000
75%       1800.000000
max      14521.000000
Name: house_size, dtype: float64

In [11]:
df = df[df["house_size"].notna()]


In [12]:
df["house_size"] = df["house_size"].fillna(df["house_size"].median())


In [13]:
# df["house_size"] = (
#     df["house_size"]
#     .str.replace("sq ft", "", regex=False)
#     .str.strip()
#     .astype(float)
# )


AttributeError: Can only use .str accessor with string values!

In [14]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")


In [15]:
df["numBalconies"] = df["numBalconies"].fillna(0)
df["isNegotiable"] = df["isNegotiable"].fillna("Unknown")


In [16]:
df = df.drop(columns=["description"], errors="ignore")


In [17]:
Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)
IQR = Q3 - Q1

df = df[(df["price"] >= Q1 - 1.5*IQR) & (df["price"] <= Q3 + 1.5*IQR)]


In [18]:
df["price_per_sqft"] = df["price"] / df["house_size"]


In [19]:
def size_bucket(size):
    if size < 500:
        return "Small"
    elif size <= 1000:
        return "Medium"
    else:
        return "Large"

df["size_bucket"] = df["house_size"].apply(size_bucket)


In [20]:
df.shape
df.info()
df.head()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 11374 entries, 0 to 13909
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   house_type        11374 non-null  object 
 1   house_size        11374 non-null  int64  
 2   location          11374 non-null  object 
 3   city              11374 non-null  object 
 4   latitude          11374 non-null  float64
 5   longitude         11374 non-null  float64
 6   price             11374 non-null  int64  
 7   currency          11374 non-null  object 
 8   numBathrooms      11320 non-null  float64
 9   numBalconies      11374 non-null  float64
 10  isNegotiable      11374 non-null  object 
 11  priceSqFt         0 non-null      float64
 12  verificationDate  11374 non-null  object 
 13  SecurityDeposit   11374 non-null  object 
 14  Status            11374 non-null  object 
 15  price_per_sqft    11374 non-null  float64
 16  size_bucket       11374 non-null  object 
dty

house_type              0
house_size              0
location                0
city                    0
latitude                0
longitude               0
price                   0
currency                0
numBathrooms           54
numBalconies            0
isNegotiable            0
priceSqFt           11374
verificationDate        0
SecurityDeposit         0
Status                  0
price_per_sqft          0
size_bucket             0
dtype: int64

In [21]:
df["property_id"] = range(1, len(df) + 1)


In [22]:
# Which city generates the highest average rent?

# Which city has the best price per sqft?

# Which locations are premium?

# Which size bucket is most profitable?

# Furnished vs unfurnished — revenue impact?

# Does negotiable pricing reduce revenue?

# Top 10 locations by revenue efficiency

In [23]:
df.to_csv("real_estate_clean.csv", index=False)
